In [1]:
from policyengine_us import Simulation
from policyengine_core.reforms import Reform
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from policyengine_core.charts import format_fig

/Users/daphnehansell/miniconda3/envs/policyengine/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
reform = Reform.from_dict(
    {
        "gov.contrib.aca.ptc_additional_bracket.in_effect": {
            "2026-01-01.2100-12-31": True
        }
    },
    country_id="us",
)



In [3]:

situation_new_york = {
  "people": {
    "you": {
      "age": {
        "2026": 30
      }
    },
    "your partner": {
      "age": {
        "2026": 30
      }
    },
    "your first dependent": {
      "age": {
        "2026": 3
      }
    }
  },
  "families": {
    "your family": {
      "members": [
        "you",
        "your partner",
        "your first dependent"
      ]
    }
  },
  "spm_units": {
    "your household": {
      "members": [
        "you",
        "your partner",
        "your first dependent"
      ]
    }
  },
  "tax_units": {
    "your tax unit": {
      "members": [
        "you",
        "your partner",
        "your first dependent"
      ]
    }
  },
  "households": {
   "your household": {
     "members": [
       "you",
       "your partner", 
       "your first dependent"  # All live in the same household
     ],
     "state_name": {
       "2026": "NY"  # Located in New York state
     },
    "county_fips": {
      "2026": "36061"
    }
   }
 },
  "marital_units": {
    "your marital unit": {
      "members": [
        "you",
        "your partner"
      ]
    },
    "your first dependent's marital unit": {
      "members": [
        "your first dependent"
      ],
      "marital_unit_id": {
        "2026": 1
      }
    }
  },
  "axes": [
    [
      {
        "name": "employment_income",
        "count": 800,
        "min": 0,
        "max": 400000
      }
    ]
  ]
}



In [4]:
situation_texas = {
  "people": {
    "you": {
      "age": {
        "2026": 25
      }
    },
    "your partner": {
      "age": {
        "2026": 28
      }
    }
  },
  "families": {
    "your family": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "spm_units": {
    "your household": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "tax_units": {
    "your tax unit": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "households": {
    "your household": {
      "members": [
        "you",
        "your partner"
      ],
      "state_name": {
        "2026": "TX"
      },
      "county_fips": {
        "2026": "48015"
      }
    }
  },
  "marital_units": {
    "your marital unit": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "axes": [
    [
      {
        "name": "employment_income",
        "count": 800,
        "min": 0,
        "max": 400000
      }
    ]
  ]
}



In [5]:
simulation_new_york = Simulation(
    situation=situation_new_york,
)
simulation_texas = Simulation(
    situation=situation_texas,
)
reformed_simulation_new_york = Simulation(
    situation=situation_new_york,
    reform=reform,
)
reformed_simulation_texas = Simulation(
    situation=situation_texas,
    reform=reform,
)

In [6]:
import copy
import pandas as pd
from policyengine_us import Simulation

PERIOD = 2026

# ------------------------------------------------------------------
# Helper: get the tax unit's 2026 FPG from the situation
# ------------------------------------------------------------------
def get_tax_unit_fpg(base_situation: dict, period=PERIOD) -> float:
    """Return the tax unit FPG for the given situation/year (first tax unit)."""
    sit = copy.deepcopy(base_situation)
    sit.pop("axes", None)

    # Ensure income isn't interfering (FPG shouldn't depend on income, but be safe)
    for person in sit["people"].values():
        person.setdefault("employment_income", {})
        person["employment_income"][str(period)] = 0

    sim = Simulation(situation=sit)
    fpg = sim.calculate("tax_unit_fpg", map_to="tax_unit", period=period)[0]
    return float(fpg)

# ------------------------------------------------------------------
# 1) Convenience: run a one-income Texas couple simulation (unchanged)
# ------------------------------------------------------------------
def calc_ptc_for_income(base_situation: dict, income: float, *, use_reform=False, period=PERIOD):
    """
    Return ACA PTC (household-level) for the given income and year.
    """
    sit = copy.deepcopy(base_situation)
    sit.pop("axes", None)

    # Split income 50/50 between the two adults
    for person in ["you", "your partner"]:
        sit["people"][person]["employment_income"] = {str(period): income / 2}

    sim = Simulation(situation=sit, reform=reform if use_reform else None)
    return sim.calculate("aca_ptc", map_to="household", period=period)[0]

# ------------------------------------------------------------------
# 2) Build targets from model-derived FPG and compute PTCs
# ------------------------------------------------------------------
fpg_2026 = get_tax_unit_fpg(situation_texas, period=PERIOD)

percent_targets = {
    "138 % FPL": 1.38,
    "300 % FPL": 3.00,
    "400 % FPL": 4.00,
}

rows = []
for label, mult in percent_targets.items():
    inc = round(fpg_2026 * mult, 2)
    rows.append({
        "income_label": f"{label} (${inc:,.2f})",
        "income_usd": inc,
        "ptc_baseline":   calc_ptc_for_income(situation_texas, inc, use_reform=False, period=PERIOD),
        "ptc_ira_reform": calc_ptc_for_income(situation_texas, inc, use_reform=True,  period=PERIOD),
    })

ptc_df = pd.DataFrame(rows)
ptc_df


,income_label,income_usd,ptc_baseline,ptc_ira_reform
0,"138 % FPL ($29,897.10)",29897.10,9268.483398,10354.979492
1,"300 % FPL ($64,993.70)",64993.70,3881.606934,6273.375000
2,"400 % FPL ($86,658.27)",86658.27,0.000000,1377.181641


In [7]:
import copy
import pandas as pd
from policyengine_us import Simulation

# ------------------------------------------------------------------
# 1.  Helper: run a one-point simulation and collect metrics
# ------------------------------------------------------------------
def get_metrics_for_income(base_situation: dict, income: float):
    """
    Returns baseline & reform PTC plus baseline Medicaid and CHIP metrics
    for a New York family of three at the specified annual income.

    Parameters
    ----------
    base_situation : dict
        Your `situation_ny` dictionary.
    income : float
        Total household employment income to test (USD, annual).

    Returns
    -------
    dict  with keys
        ptc_baseline       – ACA PTC in baseline
        ptc_ira_reform     – ACA PTC under IRA-style reform
        medicaid_cost      – household Medicaid benefit (baseline)
        per_capita_chip    – CHIP benefit ÷ household size (baseline)
    """
    # ---------------- Copy & inject the income --------------------
    sit = copy.deepcopy(base_situation)
    sit.pop("axes", None)                # single-point sim only

    # Split income equally between both adults
    for person in ["you", "your partner"]:
        sit["people"][person]["employment_income"] = {"2026": income / 2}

    hh_size = len(sit["people"])

    # ---------------- Run simulations ----------------------------
    sim_base   = Simulation(situation=sit)
    sim_reform = Simulation(situation=sit, reform=reform)

    # ---------------- Pull variables -----------------------------
    # ACA PTC
    ptc_base   = sim_base.calculate("aca_ptc",   map_to="household", period=2026)[0]
    ptc_reform = sim_reform.calculate("aca_ptc", map_to="household", period=2026)[0]
    SLCSP = sim_base.calculate("slcsp", map_to="household", period=2026)[0]

    # Medicaid benefit (adult or child)
    medicaid_cost = sim_base.calculate("medicaid_cost", map_to="household", period=2026)[0]

    # CHIP benefit – variable names differ by PE-US version:
    #   * If your build has `chip_cost`, use that.
    #   * Otherwise use `chip` (total CHIP dollars) or adjust as needed.
    chip_total = sim_base.calculate("per_capita_chip", map_to="household", period=2026)[0]
    per_capita_chip = chip_total / hh_size if hh_size else 0

    return dict(
        ptc_baseline     = ptc_base,
        ptc_ira_reform   = ptc_reform,
        medicaid_cost    = medicaid_cost,
        per_capita_chip  = per_capita_chip,
        SLCSP = SLCSP
    )

# ------------------------------------------------------------------
# 2.  Income targets (family of 3, 2026 FPL thresholds you supplied)
# ------------------------------------------------------------------
targets_ny = {
    "154 % FPL ($41,041)" : 41_041,
    "200 % FPL ($53,300)" : 53_300,
    "300 % FPL ($79,950)": 79_950,
    "405 % FPL ($107,933)": 107_933,
}

rows = []
for label, inc in targets_ny.items():
    metrics = get_metrics_for_income(situation_new_york, inc)
    rows.append(
        dict(income_label = label, income_usd = inc, **metrics)
    )

ny_ptc_df = pd.DataFrame(rows)
ny_ptc_df


,income_label,income_usd,ptc_baseline,ptc_ira_reform,medicaid_cost,per_capita_chip,SLCSP
0,"154 % FPL ($41,041)",41041,0.000000,0.000000,16480.696289,0.000000,0.000000
1,"200 % FPL ($53,300)",53300,0.000000,0.000000,12930.671875,829.929932,0.000000
2,"300 % FPL ($79,950)",79950,14042.517578,17208.537109,0.000000,829.929932,22005.537109
3,"405 % FPL ($107,933)",107933,0.000000,10996.370117,0.000000,829.929932,0.000000


In [8]:
household_income_new_york = simulation_new_york.calculate("employment_income", map_to="household", period=2026)
baseline_new_york_per_capita_chip = simulation_new_york.calculate("per_capita_chip", map_to="household", period=2026)
baseline_new_york_aca_ptc = simulation_new_york.calculate("aca_ptc", map_to="household", period=2026)
baseline_new_york_medicaid_cost = simulation_new_york.calculate("medicaid_cost", map_to="household", period=2026)
baseline_new_york_net_income_including_health_benefits = simulation_new_york.calculate("household_net_income_including_health_benefits", map_to="household", period=2026)
baseline_new_york_slcsp = simulation_new_york.calculate("slcsp", map_to="household", period=2026)

reform_new_york_per_capita_chip = reformed_simulation_new_york.calculate("per_capita_chip", map_to="household", period=2026)
reform_new_york_aca_ptc = reformed_simulation_new_york.calculate("aca_ptc", map_to="household", period=2026)
reform_new_york_medicaid_cost = reformed_simulation_new_york.calculate("medicaid_cost", map_to="household", period=2026)
reform_new_york_net_income_including_health_benefits = reformed_simulation_new_york.calculate("household_net_income_including_health_benefits", map_to="household", period=2026)





# Get household-level values for Texas
household_income_texas = simulation_texas.calculate("employment_income", map_to="household", period=2026)
baseline_texas_per_capita_chip = simulation_texas.calculate("per_capita_chip", map_to="household", period=2026)
baseline_texas_aca_ptc = simulation_texas.calculate("aca_ptc", map_to="household", period=2026)
baseline_texas_medicaid_cost = simulation_texas.calculate("medicaid_cost", map_to="household", period=2026)
baseline_texas_net_income_including_health_benefits = simulation_texas.calculate("household_net_income_including_health_benefits", map_to="household", period=2026)
baseline_texas_slcsp = simulation_texas.calculate("slcsp", map_to="household", period=2026)

reform_texas_per_capita_chip = reformed_simulation_texas.calculate("per_capita_chip", map_to="household", period=2026)
reform_texas_aca_ptc = reformed_simulation_texas.calculate("aca_ptc", map_to="household", period=2026)
reform_texas_medicaid_cost = reformed_simulation_texas.calculate("medicaid_cost", map_to="household", period=2026)
reform_texas_net_income_including_health_benefits = reformed_simulation_texas.calculate("household_net_income_including_health_benefits", map_to="household", period=2026)


# Calculate total benefits for each scenario
baseline_new_york_total = [sum(x) for x in zip(baseline_new_york_per_capita_chip, baseline_new_york_aca_ptc, baseline_new_york_medicaid_cost)]
reform_new_york_total = [sum(x) for x in zip(reform_new_york_per_capita_chip, reform_new_york_aca_ptc, reform_new_york_medicaid_cost)]

baseline_texas_total = [sum(x) for x in zip(baseline_texas_per_capita_chip, baseline_texas_aca_ptc, baseline_texas_medicaid_cost)]
reform_texas_total = [sum(x) for x in zip(reform_texas_per_capita_chip, reform_texas_aca_ptc, reform_texas_medicaid_cost)]



In [9]:
GRAY = "#808080"
BLUE_PRIMARY = "#2C6496"
TEAL_ACCENT = "#39C6C0"
DARK_GRAY = "#616161"

In [10]:
# Create NY graph
fig_new_york = go.Figure()

# Add baseline traces (solid lines)
fig_new_york.add_trace(go.Scatter(
    x=household_income_new_york, 
    y=baseline_new_york_per_capita_chip, 
    mode='lines', 
    name='CHIP (Baseline)', 
    line=dict(color=GRAY, width=2)
))

fig_new_york.add_trace(go.Scatter(
    x=household_income_new_york, 
    y=baseline_new_york_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Baseline)', 
    line=dict(color=BLUE_PRIMARY, width=2)
))

fig_new_york.add_trace(go.Scatter(
    x=household_income_new_york, 
    y=baseline_new_york_medicaid_cost, 
    mode='lines', 
    name='Medicaid (Baseline)', 
    line=dict(color=TEAL_ACCENT, width=2)
))

# Add reform traces (dotted lines)
fig_new_york.add_trace(go.Scatter(
    x=household_income_new_york, 
    y=reform_new_york_per_capita_chip, 
    mode='lines', 
    name='CHIP (Reform)', 
    line=dict(color=GRAY, width=2, dash='dot')
))

fig_new_york.add_trace(go.Scatter(
    x=household_income_new_york, 
    y=reform_new_york_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Reform)', 
    line=dict(color=BLUE_PRIMARY, width=2, dash='dot')
))

fig_new_york.add_trace(go.Scatter(
    x=household_income_new_york, 
    y=reform_new_york_medicaid_cost, 
    mode='lines', 
    name='Medicaid (Reform)', 
    line=dict(color=TEAL_ACCENT, width=2, dash='dot')
))

# Add total lines
fig_new_york.add_trace(go.Scatter(
    x=household_income_new_york, 
    y=baseline_new_york_total, 
    mode='lines', 
    name='Total Benefits (Baseline)', 
    line=dict(color=DARK_GRAY, width=2)
))

fig_new_york.add_trace(go.Scatter(
    x=household_income_new_york, 
    y=reform_new_york_total, 
    mode='lines', 
    name='Total Benefits (Reform)', 
    line=dict(color=DARK_GRAY, width=2, dash='dot')
))

# Update layout
fig_new_york.update_layout(
    title='New York Household (Family of 3) - Program Benefits by Income Level',
    xaxis_title='Household Income',
    yaxis_title='Benefit Amount',
    legend_title='Programs',
    xaxis=dict(tickformat='$,.0f', range=[0, 400000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000
)

# Create Texas graph
fig_texas = go.Figure()

# Add baseline traces (solid lines)

fig_texas.add_trace(go.Scatter(
    x=household_income_texas, 
    y=baseline_texas_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Baseline)', 
    line=dict(color=BLUE_PRIMARY, width=2)
))



fig_texas.add_trace(go.Scatter(
    x=household_income_texas, 
    y=reform_texas_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Reform)', 
    line=dict(color=BLUE_PRIMARY, width=2, dash='dot')
))



# Add total lines
fig_texas.add_trace(go.Scatter(
    x=household_income_texas, 
    y=baseline_texas_total, 
    mode='lines', 
    name='Total Benefits (Baseline)', 
    line=dict(color=DARK_GRAY, width=2)
))

fig_texas.add_trace(go.Scatter(
    x=household_income_texas, 
    y=reform_texas_total, 
    mode='lines', 
    name='Total Benefits (Reform)', 
    line=dict(color=DARK_GRAY, width=2, dash='dot')
))

# Update layout
fig_texas.update_layout(
    title='Texas Household (Couple) - Program Benefits by Income Level',
    xaxis_title='Household Income',
    yaxis_title='Benefit Amount',
    legend_title='Programs',
    xaxis=dict(tickformat='$,.0f', range=[0, 200000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000
)

# Apply your format_fig function if it exists
# If you don't have this function defined, you can remove these lines
fig_new_york = format_fig(fig_new_york)
fig_texas = format_fig(fig_texas)

# Display the figures
fig_new_york.show()
fig_texas.show()

In [11]:
#House hold net income graphs
import plotly.graph_objects as go

# ---------- NY fam  ----------
fig_ny = go.Figure()

# Baseline (solid)
fig_ny.add_trace(go.Scatter(
    x=household_income_new_york,
    y=baseline_new_york_net_income_including_health_benefits,
    mode='lines',
    name='Health Net Income (Baseline)',
    line=dict(color=DARK_GRAY, width=2)        # use your palette constant
))

# Reform (dotted)
fig_ny.add_trace(go.Scatter(
    x=household_income_new_york,
    y=reform_new_york_net_income_including_health_benefits,
    mode='lines',
    name='Health Net Income (Reform)',
    line=dict(color=DARK_GRAY, width=2, dash='dot')
))

# Layout
fig_ny.update_layout(
    title='New York Household (Family of 3) – Health-Adjusted Net Income by Household Income',
    xaxis_title='Household Income',
    yaxis_title='Health-Adjusted Net Income',
    legend_title='Scenario',
    xaxis=dict(tickformat='$,.0f', range=[0, 400_000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000
)

# Optional wrapper if you use one elsewhere
fig_ny = format_fig(fig_ny)

fig_ny.show()

# --- Δ Health-adjusted net income (Reform – Baseline) ---
delta_ny = (
    reform_new_york_net_income_including_health_benefits
    - baseline_new_york_net_income_including_health_benefits
)

fig_delta_ny = go.Figure()

fig_delta_ny.add_trace(go.Scatter(
    x=household_income_new_york,
    y=delta_ny,
    mode='lines',
    name='Δ Net Income (Reform – Baseline)',
    line=dict(color=DARK_GRAY, width=2)
))

fig_delta_ny.update_layout(
    title='New York Household (Family of 3) – Impact of Extending Enhanced Premium Tax Credits',
    xaxis_title='Household Income',
    yaxis_title='Δ Net Income',
    xaxis=dict(tickformat='$,.0f', range=[0, 400_000]),
    yaxis=dict(tickformat='$,.0f', zeroline=True, zerolinewidth=1),
    height=600,
    width=1000,
    legend=dict(orientation='h')
)

fig_delta_ny = format_fig(fig_delta_ny)  # if you use the helper elsewhere
fig_delta_ny.show()


In [12]:
# ---------- texas couple ----------
fig_tx = go.Figure()

# Baseline (solid)
fig_tx.add_trace(go.Scatter(
    x=household_income_texas,
    y=baseline_texas_net_income_including_health_benefits,
    mode='lines',
    name='Health Net Income (Baseline)',
    line=dict(color=DARK_GRAY, width=2)        # use your palette constant
))

# Reform (dotted)
fig_tx.add_trace(go.Scatter(
    x=household_income_texas,
    y=reform_texas_net_income_including_health_benefits,
    mode='lines',
    name='Health Net Income (Reform)',
    line=dict(color=DARK_GRAY, width=2, dash='dot')
))

# Layout
fig_tx.update_layout(
    title='Texas Household (Married Couple) – Health-Adjusted Net Income by Household Income',
    xaxis_title='Household Income',
    yaxis_title='Health-Adjusted Net Income',
    legend_title='Scenario',
    xaxis=dict(tickformat='$,.0f', range=[0, 200_000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000
)

# Optional wrapper if you use one elsewhere
fig_tx = format_fig(fig_tx)

fig_tx.show()
# --- Δ Health-adjusted net income (Reform – Baseline), Texas ---
delta_tx = (
    reform_texas_net_income_including_health_benefits
    - baseline_texas_net_income_including_health_benefits
)

fig_delta_tx = go.Figure()

fig_delta_tx.add_trace(go.Scatter(
    x=household_income_texas,
    y=delta_tx,
    mode='lines',
    name='Δ Net Income (Reform – Baseline)',
    line=dict(color=DARK_GRAY, width=2)
))

fig_delta_tx.update_layout(
    title='Texas Household (Family of 3) – Impact of Extending Enhanced Premium Tax Credits',
    xaxis_title='Household Income',
    yaxis_title='Δ Net Income',
    xaxis=dict(tickformat='$,.0f', range=[0, 200_000]),
    yaxis=dict(tickformat='$,.0f', zeroline=True, zerolinewidth=1),
    height=600,
    width=1000,
    legend=dict(orientation='h')
)

fig_delta_tx = format_fig(fig_delta_tx)  # if you’re using that helper
fig_delta_tx.show()



In [13]:
# ========================================================================
# CORRECT MTR CALCULATION FOR NEW YORK HOUSEHOLD
# Uses axes-based simulation for net income, then calculates MTR manually
# ========================================================================

# Step 1: Create situation with axes for vectorized net income calculation
situation_ny_for_mtr = {
    "people": {
        "you": {
            "age": {"2026": 30},
            "employment_income": {"2026": 0},  # Will be set by axes
            "self_employment_income": {"2026": 0}
        },
        "your partner": {
            "age": {"2026": 30},
            "employment_income": {"2026": 0},  # Will be set by axes
            "self_employment_income": {"2026": 0}
        },
        "your first dependent": {
            "age": {"2026": 3},
            "employment_income": {"2026": 0},
            "self_employment_income": {"2026": 0}
        }
    },
    "families": {
        "your family": {
            "members": ["you", "your partner", "your first dependent"]
        }
    },
    "spm_units": {
        "your household": {
            "members": ["you", "your partner", "your first dependent"]
        }
    },
    "tax_units": {
        "your tax unit": {
            "members": ["you", "your partner", "your first dependent"]
        }
    },
    "households": {
        "your household": {
            "members": ["you", "your partner", "your first dependent"],
            "state_name": {"2026": "NY"},
            "county_fips": {"2026": "36061"}
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you", "your partner"]
        },
        "your first dependent's marital unit": {
            "members": ["your first dependent"],
            "marital_unit_id": {"2026": 1}
        }
    },
    "axes": [[
        {
            "name": "employment_income",
            "min": 0,
            "max": 200_000,
            "count": 200,
            "period": "2026"
        }
    ]]
}

# Step 2: Calculate baseline net income using axes
sim_ny_baseline = Simulation(situation=situation_ny_for_mtr)
household_income_ny_mtr = sim_ny_baseline.calculate("employment_income", map_to="household", period=2026)
baseline_ny_net_income = sim_ny_baseline.calculate(
    "household_net_income_including_health_benefits",
    map_to="household",
    period=2026
)

# Step 3: Calculate reform net income using axes
sim_ny_reform = Simulation(situation=situation_ny_for_mtr, reform=reform)
reform_ny_net_income = sim_ny_reform.calculate(
    "household_net_income_including_health_benefits",
    map_to="household",
    period=2026
)

# Step 4: Calculate MTR from adjacent points
def calc_mtr(incomes, net_incomes):
    """Calculate MTR between adjacent income points."""
    mtrs = []
    mtr_incomes = []
    for i in range(len(incomes) - 1):
        income_change = incomes[i + 1] - incomes[i]
        net_change = net_incomes[i + 1] - net_incomes[i]
        if income_change > 0 and not np.isnan(net_incomes[i]) and not np.isnan(net_incomes[i + 1]):
            mtr = 1 - (net_change / income_change)
            mtrs.append(mtr)
            mtr_incomes.append((incomes[i] + incomes[i + 1]) / 2)
    return np.array(mtr_incomes), np.array(mtrs)

baseline_ny_mtr_incomes, baseline_ny_mtrs = calc_mtr(household_income_ny_mtr, baseline_ny_net_income)
reform_ny_mtr_incomes, reform_ny_mtrs = calc_mtr(household_income_ny_mtr, reform_ny_net_income)

# Step 5: Create the chart
fig_new_york_mtr = go.Figure()

fig_new_york_mtr.add_trace(go.Scatter(
    x=baseline_ny_mtr_incomes,
    y=np.clip(baseline_ny_mtrs, -1.0, 1.0),
    mode='lines',
    name='Baseline',
    line=dict(color=DARK_GRAY, width=2),
    hovertemplate='Income: $%{x:,.0f}<br>Baseline MTR: %{y:.1%}<extra></extra>'
))

fig_new_york_mtr.add_trace(go.Scatter(
    x=reform_ny_mtr_incomes,
    y=np.clip(reform_ny_mtrs, -1.0, 1.0),
    mode='lines',
    name='IRA Extension',
    line=dict(color=BLUE_PRIMARY, width=2, dash='dash'),
    hovertemplate='Income: $%{x:,.0f}<br>Reform MTR: %{y:.1%}<extra></extra>'
))

fig_new_york_mtr.update_layout(
    title='Marginal tax rate including health benefits - New York household',
    xaxis_title='Employment income',
    yaxis_title='Marginal tax rate',
    xaxis=dict(
        tickformat='$,.0f',
        range=[0, 200_000],
        gridcolor='lightgray',
        showgrid=True
    ),
    yaxis=dict(
        tickformat='.0%',
        range=[-1.0, 1.0],
        gridcolor='lightgray',
        showgrid=True,
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='gray'
    ),
    height=600,
    width=1000,
    hovermode='x unified',
    plot_bgcolor='white',
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig_new_york_mtr = format_fig(fig_new_york_mtr)
fig_new_york_mtr.show()

In [14]:
# ========================================================================
# CORRECT MTR CALCULATION FOR TEXAS COUPLE
# Uses axes-based simulation for net income, then calculates MTR manually
# ========================================================================

# Step 1: Create situation with axes for vectorized net income calculation
situation_tx_for_mtr = {
    "people": {
        "you": {
            "age": {"2026": 25},
            "employment_income": {"2026": 0},  # Will be set by axes
            "self_employment_income": {"2026": 0}
        },
        "your partner": {
            "age": {"2026": 28},
            "employment_income": {"2026": 0},  # Will be set by axes
            "self_employment_income": {"2026": 0}
        }
    },
    "families": {
        "your family": {
            "members": ["you", "your partner"]
        }
    },
    "spm_units": {
        "your household": {
            "members": ["you", "your partner"]
        }
    },
    "tax_units": {
        "your tax unit": {
            "members": ["you", "your partner"]
        }
    },
    "households": {
        "your household": {
            "members": ["you", "your partner"],
            "state_name": {"2026": "TX"},
            "county_fips": {"2026": "48015"}
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you", "your partner"]
        }
    },
    "axes": [[
        {
            "name": "employment_income",
            "min": 0,
            "max": 200_000,
            "count": 200,
            "period": "2026"
        }
    ]]
}

# Step 2: Calculate baseline net income using axes
sim_tx_baseline = Simulation(situation=situation_tx_for_mtr)
household_income_tx_mtr = sim_tx_baseline.calculate("employment_income", map_to="household", period=2026)
baseline_tx_net_income = sim_tx_baseline.calculate(
    "household_net_income_including_health_benefits",
    map_to="household",
    period=2026
)

# Step 3: Calculate reform net income using axes
sim_tx_reform = Simulation(situation=situation_tx_for_mtr, reform=reform)
reform_tx_net_income = sim_tx_reform.calculate(
    "household_net_income_including_health_benefits",
    map_to="household",
    period=2026
)

# Step 4: Calculate MTR from adjacent points (reuse function from NY cell)
baseline_tx_mtr_incomes, baseline_tx_mtrs = calc_mtr(household_income_tx_mtr, baseline_tx_net_income)
reform_tx_mtr_incomes, reform_tx_mtrs = calc_mtr(household_income_tx_mtr, reform_tx_net_income)

# Step 5: Create the chart
fig_texas_mtr = go.Figure()

fig_texas_mtr.add_trace(go.Scatter(
    x=baseline_tx_mtr_incomes,
    y=np.clip(baseline_tx_mtrs, -1.0, 1.0),
    mode='lines',
    name='Baseline',
    line=dict(color=DARK_GRAY, width=2),
    hovertemplate='Income: $%{x:,.0f}<br>Baseline MTR: %{y:.1%}<extra></extra>'
))

fig_texas_mtr.add_trace(go.Scatter(
    x=reform_tx_mtr_incomes,
    y=np.clip(reform_tx_mtrs, -1.0, 1.0),
    mode='lines',
    name='IRA Extension',
    line=dict(color=BLUE_PRIMARY, width=2, dash='dash'),
    hovertemplate='Income: $%{x:,.0f}<br>Reform MTR: %{y:.1%}<extra></extra>'
))

fig_texas_mtr.update_layout(
    title='Marginal tax rate including health benefits - Texas couple',
    xaxis_title='Employment income',
    yaxis_title='Marginal tax rate',
    xaxis=dict(
        tickformat='$,.0f',
        range=[0, 200_000],
        gridcolor='lightgray',
        showgrid=True
    ),
    yaxis=dict(
        tickformat='.0%',
        range=[-1.0, 1.0],
        gridcolor='lightgray',
        showgrid=True,
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='gray'
    ),
    height=600,
    width=1000,
    hovermode='x unified',
    plot_bgcolor='white',
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig_texas_mtr = format_fig(fig_texas_mtr)
fig_texas_mtr.show()